# Лабораторная работа №5

In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет со свойствами различных грибов, целевым параметром является съедобность или несъедобность гриба.

In [14]:
mushrooom_dataset = pd.read_csv('MushroomDataset/secondary_data.csv',delimiter=';')

Выделяем таргетный столбец в отдельный Series, а все остальные столбы в DataFrame. Также удаляем все столбцы, где большая часть данных типа NaN и заменяем все классы на номер буквы в алфавите.

In [15]:
y = mushrooom_dataset['class']
X = mushrooom_dataset.drop('class',axis=1)
X = X.drop(columns=['cap-surface','gill-spacing','gill-attachment','stem-root',
                    'stem-surface','veil-type','veil-color','ring-type','spore-print-color'])
X.replace({'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,
           'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,
           'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,
           't':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26},inplace=True)

После устранения пропусков нормализуем все значения.

In [16]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем данные на тренировочную и тестовую части.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.2)

### Дерево решений

Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметров criterion, splitter и max_depth, которые отвечает за функция оценки качества, стратегия разделения и максимальная грубина дерева.

In [18]:
des_tree_classifier = DecisionTreeClassifier()
des_tree_params ={
    "criterion": ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth':range(1,100),
}
des_tree_grid = GridSearchCV(des_tree_classifier, des_tree_params, cv=2, n_jobs=-1,verbose=3)
des_tree_grid.fit(X_train, y_train)
best_des_tree_params = des_tree_grid.best_params_

Fitting 2 folds for each of 594 candidates, totalling 1188 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [19]:
best_des_tree_model = DecisionTreeClassifier(**best_des_tree_params)
best_des_tree_model.fit(X_train, y_train)
des_tree_predicted = best_des_tree_model.predict(X_test)
print('Used params:',best_des_tree_params)
print('Evaluation:\n', metrics.classification_report(y_test, des_tree_predicted,digits=5))

Used params: {'criterion': 'entropy', 'max_depth': 35, 'splitter': 'random'}
Evaluation:
               precision    recall  f1-score   support

           e    0.98919   0.98738   0.98828      5466
           p    0.98979   0.99126   0.99052      6748

    accuracy                        0.98952     12214
   macro avg    0.98949   0.98932   0.98940     12214
weighted avg    0.98952   0.98952   0.98952     12214



### Случайный лес

Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметров n_estimators, criterion и max_depth, которыt отвечают за кол-во деревьев, критерий оценки качества и максимальная глубина деревьев.

In [20]:
rand_forest_classifier = RandomForestClassifier()
rand_forest_params ={
    'n_estimators': np.geomspace(1,1e2,10,dtype=int),
    'criterion': ['gini','entropy','log_loss'],
    'max_depth':range(1,100)
}
rand_forest_grid = GridSearchCV(rand_forest_classifier, rand_forest_params, cv=2, n_jobs=-1,verbose=3)
rand_forest_grid.fit(X_train, y_train)
rand_forest_best_params = rand_forest_grid.best_params_

Fitting 2 folds for each of 2970 candidates, totalling 5940 fits


После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [21]:
rand_forest_best_model = RandomForestClassifier(**rand_forest_best_params)
rand_forest_best_model.fit(X_train, y_train)
rand_forest_predicted = rand_forest_best_model.predict(X_test)
print('Used params:',rand_forest_best_params)
print('Evaluation:\n', metrics.classification_report(y_test, rand_forest_predicted,digits=5))

Used params: {'criterion': 'entropy', 'max_depth': 81, 'n_estimators': 100}
Evaluation:
               precision    recall  f1-score   support

           e    0.99343   0.99579   0.99461      5466
           p    0.99659   0.99467   0.99562      6748

    accuracy                        0.99517     12214
   macro avg    0.99501   0.99523   0.99512     12214
weighted avg    0.99517   0.99517   0.99517     12214

